In [1]:
import pandas as pd
import numpy as np
from typing import Dict, List, Any, Optional, Union
from dataclasses import dataclass
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, classification_report
import json
import logging
from datetime import datetime
import traceback
import pprint
import re

# Дополнительные импорты для простого поиска
import requests
from bs4 import BeautifulSoup
import urllib.parse

# LangChain imports
from langchain.schema import BaseMessage, HumanMessage, SystemMessage
from langchain.chat_models.base import BaseChatModel
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_community.chat_models import ChatOllama

# ScrapeGraphAI imports заменяем на Tavily
try:
    from langchain_community.tools.tavily_search import TavilySearchResults
except ImportError:
    print("Предупреждение: TavilySearchResults не доступен. Установите: pip install langchain-community")
    TavilySearchResults = None

# LangGraph imports
from langgraph.graph import StateGraph, END
# from langgraph.prebuilt import ToolNode
from typing_extensions import TypedDict
from typing import cast

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

d:\Notebooks\PyTorch\.env\Lib\site-packages\langchain_community\chat_models\ollama.py:20: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.llms.ollama import OllamaEndpointNotFoundError, _OllamaCommon
d:\Notebooks\PyTorch\.env\Lib\site-packages\pydantic\_internal\_generate_schema.py:918: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `TavilySearchAPIWrapper` to V2.
  warn(


In [2]:
# Загрузка данных
PATH = 'data_final_for_dls.jsonl'
PATH_NEW = 'data_final_for_dls_new_new.jsonl'
data = pd.read_json(path_or_buf=PATH_NEW, lines=True)

train_data = data[570:]
eval_data = data[:570]
eval_data = eval_data[eval_data["relevance"] != 0.1]

eval_data = eval_data.fillna("")

In [3]:

@dataclass
class CompanyData:
    """Структура данных о компании"""
    text: str  # Пользовательский запрос
    address: str
    name: str
    normalized_main_rubric_name_ru: str
    permalink: int
    prices_summarized: str
    reviews_summarized: str
    relevance_new: float  # Правильный ответ для оценки

class AgentState(TypedDict):
    """Состояние агента"""
    company_data: CompanyData
    user_query: str
    company_info: str
    search_results: Optional[str]
    preliminary_assessment: Optional[str]
    final_relevance_score: Optional[float]
    reasoning: Optional[str]
    messages: List[BaseMessage]
    needs_search: Optional[bool] = None

class RelevanceAgent:
    """Агент для оценки релевантности организаций"""
    
    def __init__(self, 
                 llm: BaseChatModel,
                 use_search_tool: bool = True,
                 search_threshold: float = 0.5,
                 search_tool_type: str = "tavily",
                 tavily_api_key: Optional[str] = None):
        """
        Инициализация агента
        
        Args:
            llm: Языковая модель
            use_search_tool: Использовать ли инструмент поиска
            search_threshold: Порог для использования поиска (если предварительная оценка неопределенная)
            search_tool_type: Тип поиска ("tavily" или "simple")
            tavily_api_key: API ключ для Tavily (1000 запросов бесплатно в месяц)
        """
        self.llm = llm
        self.use_search_tool = use_search_tool
        self.search_threshold = search_threshold
        self.search_tool_type = search_tool_type
        
        # Инициализация инструмента поиска
        if use_search_tool and search_tool_type == "tavily":
            if TavilySearchResults is None:
                raise ImportError("TavilySearchResults не доступен. Установите: pip install langchain-community")
            
            # Tavily предоставляет 1000 бесплатных запросов в месяц!
            self.search_tool = TavilySearchResults(
                max_results=3,
                api_key=tavily_api_key  # Если None, будет использован TAVILY_API_KEY из переменных окружения
            )
        elif use_search_tool and search_tool_type == "simple":
            # Простой поиск без внешних API
            self.search_tool = None
        else:
            self.search_tool = None
        
        # Создание графа агента
        self.graph = self._create_graph()
    
    def _create_graph(self) -> StateGraph:
        """Создание графа агента"""
        workflow = StateGraph(AgentState)
        
        # Добавление узлов
        workflow.add_node("prepare_data", self._prepare_data)
        workflow.add_node("run_preliminary_assessment", self._preliminary_assessment)
        workflow.add_node("decide_search", self._decide_search)
        workflow.add_node("search_additional_info", self._search_additional_info)
        workflow.add_node("final_assessment", self._final_assessment)
        
        # Добавление рёбер
        workflow.set_entry_point("prepare_data")
        workflow.add_edge("prepare_data", "run_preliminary_assessment")
        workflow.add_edge("run_preliminary_assessment", "decide_search")
        workflow.add_conditional_edges(
            "decide_search",
            self._should_search,
            {
                "search": "search_additional_info",
                "final": "final_assessment"
            }
        )
        workflow.add_edge("search_additional_info", "final_assessment")
        workflow.add_edge("final_assessment", END)
        
        return workflow.compile()
    
    def _prepare_data(self, state: AgentState) -> AgentState:
        """Подготовка данных о компании"""
        company_data = state["company_data"]
        
        company_info = f"""
        Название: {company_data.name}
        Адрес: {company_data.address}
        Рубрика: {company_data.normalized_main_rubric_name_ru}
        Цены: {company_data.prices_summarized}
        Отзывы: {company_data.reviews_summarized}
        """
        
        state["company_info"] = company_info
        state["user_query"] = company_data.text
        
        return state
    
    def _preliminary_assessment(self, state: AgentState) -> AgentState:
        """Предварительная оценка релевантности"""
        system_prompt = """
        Ты эксперт по оценке релевантности организаций пользовательским запросам.
        
        Твоя задача - провести предварительную оценку релевантности организации запросу пользователя.
        
        Оцени следующие аспекты:
        1. Соответствие типа деятельности организации запросу
        2. Географическое соответствие (если указано в запросе)
        3. Соответствие ценовой категории (если указано в запросе)
        4. Качество услуг по отзывам (если релевантно для запроса)
        
        Ответь в формате JSON:
        {
            "confidence": float (0.0-1.0),
            "preliminary_score": float (0.0-1.0),
            "reasoning": "подробное объяснение",
            "needs_additional_search": boolean
        }
        """
        
        user_prompt = f"""
        Пользовательский запрос: {state["user_query"]}
        
        Информация об организации:
        {state["company_info"]}
        
        Проведи предварительную оценку релевантности.
        """
        
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_prompt)
        ]
        response = self.llm.invoke(messages)

        content = response.content
        assessment = safe_json_parse(content)
        if assessment is None:
            logger.warning("Не удалось распарсить JSON ответ, используем fallback")
            print(response.content.lower())
            assessment = {
                "confidence": 0.5,
                "preliminary_score": 0.5,
                "reasoning": content,
                "needs_additional_search": True
            }
        
        state["preliminary_assessment"] = assessment
        
        return state
    
    def _decide_search(self, state: AgentState) -> AgentState:
        """Решение о необходимости дополнительного поиска"""
        assessment = state["preliminary_assessment"]
        
        # Используем поиск если:
        # 1. Модель явно указала на необходимость
        # 2. Уверенность низкая
        # 3. Включен инструмент поиска
        needs_search = (
            self.use_search_tool and 
            (assessment.get("needs_additional_search", False) or 
             assessment.get("confidence", 0) < self.search_threshold)
        )
        # print(f"Это решение из _decide_search {needs_search}")
        state["needs_search"] = needs_search
        return state
    
    def _should_search(self, state: AgentState) -> str:
        """Условное ребро для решения о поиске"""
        # print(f"[CONDITIONAL] needs_search: {state.get('needs_search')}")
        return "search"  if state.get("needs_search", False) else "final"
    
    def _search_additional_info(self, state: AgentState) -> AgentState:
        """Поиск дополнительной информации"""
        company_data = state["company_data"]
        
        # Формируем поисковый запрос
        search_query = f"{company_data.name} {company_data.address}"
        
        try:
            if self.search_tool_type == "tavily":
                # Используем TavilySearchResults (1000 бесплатных запросов в месяц)
                search_results = self.search_tool.invoke({"query": search_query})
                # Форматируем результаты Tavily
                formatted_results = self._format_tavily_results(search_results)
                state["search_results"] = formatted_results
            elif self.search_tool_type == "simple":
                # Простой поиск без внешних API
                search_results = self._simple_search(search_query)
                state["search_results"] = search_results
            else:
                state["search_results"] = "Поиск отключен"

            
                
        except Exception as e:
            logger.error(f"Ошибка при поиске: {e}")
            state["search_results"] = "Не удалось получить дополнительную информацию"
        
        return state
    
    def _format_tavily_results(self, results: List[Dict]) -> str:
        """Форматирование результатов Tavily для LLM"""
        if not results:
            return "Результаты поиска не найдены"
        
        formatted = []
        for i, result in enumerate(results[:3], 1):  # Берем первые 3 результата
            title = result.get('title', 'Без названия')
            content = result.get('content', '')
            url = result.get('url', '')
            
            formatted.append(f"Результат {i}:")
            formatted.append(f"Заголовок: {title}")
            formatted.append(f"Содержание: {content[:300]}...")  # Ограничиваем длину
            formatted.append(f"URL: {url}")
            formatted.append("---")
        
        return "\n".join(formatted)
    
    def _simple_search(self, query: str) -> str:
        """Простой поиск без внешних API (бесплатно)"""
        try:
            import requests
            from bs4 import BeautifulSoup
            import urllib.parse
            
            # Поиск в Google (простой вариант)
            # search_url = f"https://www.google.com/search?q={urllib.parse.quote(query)}"
            # headers = {
            #     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            # }
            search_url = f"https://html.duckduckgo.com/html/?q={urllib.parse.quote_plus(query)}"
            headers = {
                "User-Agent": "Mozilla/5.0"
            }
            
            response = requests.get(search_url, headers=headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Извлекаем результаты поиска
            results = []
            # for result in soup.find_all('div', class_='BNeawe s3v9rd AP7Wnd')[:3]:  # Берем первые 3 результата
            #     if result.text:
            #         results.append(result.text)
            for a in soup.find_all("a", class_="result__a")[:3]:
                results.append(a.get_text(strip=True))
            # print(f"Это результат поиска {results}")
            return "\n".join(results) if results else "Результаты поиска не найдены" # f"Имитированный результат поиска по запросу: {query}"
            
        except Exception as e:
            logger.error(f"Ошибка простого поиска: {e}")
            return "Не удалось выполнить поиск"
    
    def _final_assessment(self, state: AgentState) -> AgentState:
        """Финальная оценка релевантности"""
        system_prompt = """
        Ты эксперт по оценке релевантности организаций пользовательским запросам.
        
        На основе всей доступной информации дай финальную оценку релевантности организации запросу пользователя.
        
        Релевантность определяется как:
        - 1.0 (релевантно): организация полностью соответствует запросу пользователя
        - 0.0 (не релевантно): организация не соответствует запросу пользователя
        
        Ответь в формате JSON:
        {
            "relevance_score": float (0.0 или 1.0),
            "detailed_reasoning": "подробное обоснование решения"
        }
        """
        
        search_info = ""
        if state.get("search_results"):
            search_info = f"\n\nДополнительная информация из поиска:\n{state['search_results']}"
        
        user_prompt = f"""
        Пользовательский запрос: {state["user_query"]}
        
        Информация об организации:
        {state["company_info"]}
        
        Предварительная оценка: {state["preliminary_assessment"]}
        {search_info}
        
        Дай финальную оценку релевантности (0.0 или 1.0).
        """
        
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_prompt)
        ]
        
        response = self.llm.invoke(messages)

        content = response.content
        parsed = safe_json_parse(content)

        if parsed:
            state["final_relevance_score"] = parsed.get("relevance_score", 0.0)
            state["reasoning"] = parsed.get("detailed_reasoning", "")
        else:
            logger.warning("Не удалось распарсить финальный JSON ответ")
            print(content.lower())
            if "1.0" in content or "релевантно" in content:
                state["final_relevance_score"] = 1.0
            else:
                state["final_relevance_score"] = 0.0
            state["reasoning"] = content
        
        return state
    
    def evaluate_relevance(self, company_data: CompanyData) -> Dict[str, Any]:
        """Оценка релевантности одной организации"""
        initial_state = cast(AgentState, {
            # "__start__": "prepare_data",
            "company_data": company_data,
            "user_query": "",
            "company_info": "",
            "search_results": None,
            "preliminary_assessment": None,
            "final_relevance_score": None,
            "reasoning": None,
            "messages": []
        })

        final_state = self.graph.invoke(initial_state)
        
        return {
            "relevance_score": final_state["final_relevance_score"],
            "reasoning": final_state["reasoning"],
            "preliminary_assessment": final_state["preliminary_assessment"],
            "used_search": final_state.get("search_results") is not None
        }

class AgentEvaluator:
    """Класс для оценки качества работы агента"""
    
    def __init__(self, agent: RelevanceAgent):
        self.agent = agent
    
    def evaluate_on_dataset(self, df: pd.DataFrame) -> Dict[str, Any]:
        """
        Оценка агента на датасете
        
        Args:
            df: DataFrame с данными для оценки
            
        Returns:
            Словарь с метриками качества
        """
        results = []
        predictions = []
        true_labels = []
        
        logger.info(f"Начинаем оценку на {len(df)} примерах")
        
        for idx, row in df.iterrows():
            try:
                # Подготовка данных
                company_data = CompanyData(
                    text=row['Text'],
                    address=row['address'],
                    name=row['name'],
                    normalized_main_rubric_name_ru=row['normalized_main_rubric_name_ru'],
                    permalink=row['permalink'],
                    prices_summarized=row['prices_summarized'],
                    reviews_summarized=row['reviews_summarized'],
                    relevance_new=row['relevance_new']
                )
                
                # Получение предсказания
                result = self.agent.evaluate_relevance(company_data)
                
                # Сохранение результатов
                predictions.append(result["relevance_score"])
                true_labels.append(company_data.relevance_new)
                
                results.append({
                    'index': idx,
                    'prediction': result["relevance_score"],
                    'true_label': company_data.relevance_new,
                    'reasoning': result["reasoning"],
                    'used_search': result["used_search"],
                    'preliminary_assessment': result["preliminary_assessment"]
                })
                
                if (idx + 1) % 10 == 0:
                    logger.info(f"Обработано {idx + 1}/{len(df)} примеров")
                    
            except Exception as e:
                logger.error(f"Ошибка при обработке примера {idx}: {e}")
                predictions.append(0.0)  # Default prediction
                true_labels.append(row['relevance_new'])
                traceback.print_exc()
        
        # Вычисление метрик
        metrics = self._calculate_metrics(true_labels, predictions)
        
        return {
            'metrics': metrics,
            'detailed_results': results,
            'predictions': predictions,
            'true_labels': true_labels
        }
    
    def _calculate_metrics(self, true_labels: List[float], predictions: List[float]) -> Dict[str, float]:
        """Вычисление метрик качества"""
        # Конвертация в numpy arrays
        y_true = np.array(true_labels)
        y_pred = np.array(predictions)

        thresh = 0.5
        y_cls = (y_pred > thresh).astype(int)
        
        # Основные метрики
        accuracy = accuracy_score(y_true, y_cls)
        f1 = f1_score(y_true, y_cls)
        
        # ROC-AUC (если есть вероятностные предсказания)
        try:
            roc_auc = roc_auc_score(y_true, y_pred)
        except ValueError:
            # Если все предсказания одинаковые
            roc_auc = 0.5
        
        # Дополнительные метрики
        from sklearn.metrics import precision_score, recall_score
        precision = precision_score(y_true, y_cls, zero_division=0)
        recall = recall_score(y_true, y_cls, zero_division=0)
        
        return {
            'accuracy': accuracy,
            'f1_score': f1,
            'roc_auc': roc_auc,
            'precision': precision,
            'recall': recall
        }
    
    def print_evaluation_report(self, evaluation_results: Dict[str, Any]):
        """Вывод отчёта об оценке"""
        metrics = evaluation_results['metrics']
        
        print("\n" + "="*50)
        print("ОТЧЁТ ОБ ОЦЕНКЕ АГЕНТА")
        print("="*50)
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"F1-Score: {metrics['f1_score']:.4f}")
        print(f"ROC-AUC: {metrics['roc_auc']:.4f}")
        print(f"Precision: {metrics['precision']:.4f}")
        print(f"Recall: {metrics['recall']:.4f}")
        
        # Статистика по использованию поиска
        detailed_results = evaluation_results['detailed_results']
        search_usage = sum(1 for r in detailed_results if r['used_search'])
        print(f"\nИспользование поиска: {search_usage}/{len(detailed_results)} ({search_usage/len(detailed_results)*100:.1f}%)")
        
        # Анализ ошибок
        threshold = 0.5
        errors = [r for r in detailed_results if int(r['prediction'] > threshold) != r['true_label']]
        print(f"Количество ошибок: {len(errors)}")
        
        if errors:
            print("\nПримеры ошибок:")
            for i, error in enumerate(errors[:10]):  # Показываем первые 10 ошибoк
                print(f"\nОшибка {i+1}:")
                print(f"  Предсказание: {error['prediction']}, Истина: {error['true_label']}")
                print(f"  Обоснование: {error['reasoning'][:300]}...")

# Очистка выхода модели
def safe_json_parse(text: str) -> dict:
    # Убираем markdown-обёртки ```json ... ```
    text = text.strip()
    text = re.sub(r"^```json", "", text)
    text = re.sub(r"```$", "", text).strip()

    # Пробуем вырезать первое вхождение {...}
    match = re.search(r"\{.*\}", text, flags=re.DOTALL)
    if match:
        try:
            return json.loads(match.group())
        except json.JSONDecodeError:
            pass
    
    # Пробуем напрямую
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        return None

# Утилиты для создания различных LLM
def create_llm(model_type: str, **kwargs) -> BaseChatModel:
    """
    Создание языковой модели
    
    Args:
        model_type: Тип модели ('openai', 'anthropic', 'ollama')
        **kwargs: Дополнительные параметры для модели
    """
    if model_type == 'openai':
        return ChatOpenAI(
            model=kwargs.get('model', 'gpt-3.5-turbo'),
            temperature=kwargs.get('temperature', 0.1),
            **kwargs
        )
    elif model_type == 'anthropic':
        return ChatAnthropic(
            model=kwargs.get('model', 'claude-3-sonnet-20240229'),
            temperature=kwargs.get('temperature', 0.1),
            **kwargs
        )
    elif model_type == 'ollama':
        return ChatOllama(
            model=kwargs.get('model', 'gemma3'),
            temperature=kwargs.get('temperature', 0.1),
            base_url= kwargs.get('base_url', 'http://localhost:11434/v1/')

           #  **kwargs
        )
    else:
        raise ValueError(f"Неподдерживаемый тип модели: {model_type}")

In [4]:
# Запуск Агента
if __name__ == "__main__":
    # Создание LLM (замените на нужную модель)
    # llm = create_llm('openai', model='gpt-3.5-turbo')

    # llm_ollama = create_llm('mistral', model="mistral-medium", base_url="https://api.mistral.ai/v1/")
    llm_ollama = create_llm('ollama', model="deepseek-r1", base_url="http://localhost:11434") 

    # ВАРИАНТ 1: Использование TavilySearchResults (1000 бесплатных запросов в месяц)
    # Получите API ключ на https://tavily.com
    # Можно установить переменную окружения TAVILY_API_KEY или передать явно
    # agent = RelevanceAgent(
    #     llm=llm_ollama,
    #     search_threshold = 0.5,
    #     use_search_tool= True, 
    #     search_tool_type= "tavily",
    #     tavily_api_key="your_tavily_api_key",  # Или None для использования из переменных окружения,
    # )
    
    # ВАРИАНТ 2: Простой поиск (полностью бесплатно, но менее надежно)
    agent = RelevanceAgent(llm=llm_ollama, search_threshold=0.4 , use_search_tool=True, search_tool_type="simple")
    
    # ВАРИАНТ 3: Без поиска (только на основе данных из датасета)
    # agent = RelevanceAgent(llm, use_search_tool=False)
    
    # Создание оценщика
    evaluator = AgentEvaluator(agent)
    
    print("Доступные варианты поиска:")
    print("1. Tavily: 1000 бесплатных запросов в месяц (рекомендуется)")
    print("2. Simple: Полностью бесплатно, но менее надежно")
    print("3. Без поиска: Только анализ данных из датасета")

    # eval_data= eval_data[:5] # Дебаг 
    
    # Оценка агента
    results = evaluator.evaluate_on_dataset(eval_data)
    
    # Вывод отчёта
    evaluator.print_evaluation_report(results)

INFO:__main__:Начинаем оценку на 500 примерах


Доступные варианты поиска:
1. Tavily: 1000 бесплатных запросов в месяц (рекомендуется)
2. Simple: Полностью бесплатно, но менее надежно
3. Без поиска: Только анализ данных из датасета


INFO:__main__:Обработано 10/500 примеров
INFO:__main__:Обработано 20/500 примеров
INFO:__main__:Обработано 30/500 примеров
INFO:__main__:Обработано 40/500 примеров
INFO:__main__:Обработано 50/500 примеров
INFO:__main__:Обработано 60/500 примеров
INFO:__main__:Обработано 70/500 примеров
INFO:__main__:Обработано 80/500 примеров
INFO:__main__:Обработано 90/500 примеров


<think>
окей, пользователь спрашивает про гостевые дома в береговом, республика крым, феодосийский район. дает список отзывов и предварительную оценку организации "гостевой дом ларимар". 

пользователь явно хочет получить окончательное решение по релевантности этой организации для его запроса. он уже предоставил довольно подробные данные, включая название организации (разные варианты написания), адрес и цитаты из отзывов.

хм, нужно проверить соответствие организации местоположению и типу объекта. адрес указан явно: республика крым, городской округ феодосия, село береговое - это точное совпадение по всем трем параметрам. рубрика "гостиница" в контексте крыма обычно означает именно гостевые дома или мини-отели.

теперь посмотрим на отзывы: большая часть из них действительно говорит о том, что это гостевой дом (комфортное проживание, уютные номера и т.д.). хотя есть негативные комментарии про жару в номерах и отсутствие интернета - но это не меняет основной факта, что место является имен

INFO:__main__:Обработано 100/500 примеров
INFO:__main__:Обработано 120/500 примеров
INFO:__main__:Обработано 130/500 примеров
INFO:__main__:Обработано 140/500 примеров
INFO:__main__:Обработано 160/500 примеров
INFO:__main__:Обработано 170/500 примеров
INFO:__main__:Обработано 180/500 примеров
INFO:__main__:Обработано 190/500 примеров
INFO:__main__:Обработано 200/500 примеров
INFO:__main__:Обработано 210/500 примеров
INFO:__main__:Обработано 220/500 примеров
INFO:__main__:Обработано 230/500 примеров
INFO:__main__:Обработано 250/500 примеров
INFO:__main__:Обработано 260/500 примеров


...

<think>
hmm, пользователь хочет, чтобы я оценил релевантность организации "кинза" для запроса "кафе греческой кухни". интересно, почему он предоставил такой большой список отзывов? возможно, это поможет мне быть более точным в оценке.

сначала нужно понять сам запрос: кафе греческой кухни. организация называется "кинза", но её рубрика - доставка еды и обедов, а блюда перечислены как грузинская кухня (хинкали, долма и т.д.). греческая кухня и грузинская - это разные вещи, хотя обе из региона кавказа.

пользователь дал очень подробный список отзывов, но я вижу только положительные моменты о вкусе еды и атмосфере. нужно проверить, есть ли среди них упоминания о греческой кухне. видимо, нет - все отзывы про грузинскую кухню.

теперь оценю каждый аспект:
1. тип деятельности: организация предлагает еду, но это явно не греческая кухня.
2. география: адрес в москве, но запрос без указания региона, так что это нейтрально.
3. цена: отзывы упоминают цены как проблему (высокие), а не описывают их.

INFO:__main__:Обработано 500/500 примеров
INFO:__main__:Обработано 510/500 примеров
INFO:__main__:Обработано 520/500 примеров
INFO:__main__:Обработано 530/500 примеров
INFO:__main__:Обработано 540/500 примеров


<think>
окей, пользователь спрашивает про релевантность организации "цифровой канал" для запроса "домашний интернет в курске что подключить отзывы и цены". 

из предварительной оценки видно, что компания уже частично соответствует: это телекоммуникационная компания с адресом в курске. но есть два ключевых момента без которых пользователь не может принять решение - отсутствие информации о ценах и отзывах.

пользователь явно хочет сравнить варианты для выбора провайдера, но у нас нет данных по этим аспектам. хотя компания относится к телеком-сфере, без цен и отзывов это не полезно для принятия окончательного решения о подключении интернета.

интересно, что пользователь использует фразу "что подключить", что может указывать на желание сравнить несколько вариантов. но у нас нет конкурентных данных или других компаний в курске.

предварительная оценка была 0.6 из-за нехватки цен и отзывов, а дополнительный поиск не добавил новых ключевых сведений - просто подтверждает название компании и ее

INFO:__main__:Обработано 570/500 примеров



ОТЧЁТ ОБ ОЦЕНКЕ АГЕНТА
Accuracy: 0.7260
F1-Score: 0.7787
ROC-AUC: 0.7429
Precision: 0.7980
Recall: 0.7603

Использование поиска: 405/500 (81.0%)
Количество ошибок: 137

Примеры ошибок:

Ошибка 1:
  Предсказание: 0.0, Истина: 1.0
  Обоснование: Организация MaxiLife является стоматологической клиникой, предоставляющей широкий спектр услуг в области здоровья и красоты. Хотя в списке услуг организации указано наличие эпиляции (пункт №1), основное направление деятельности — это стоматология. Отзывы подтверждают предоставление косметологических...

Ошибка 2:
  Предсказание: 0.0, Истина: 1.0
  Обоснование: Организация имеет название KFC, что является частью сети быстрого питания. Однако дополнительная информация из поиска не подтверждает существования сети Rostic's или KFC в целом. Ответ описывает конкретный ресторан и его меню, но не указывает на наличие сети с множеством филиалов. Название 'Rostic's...

Ошибка 3:
  Предсказание: 0.0, Истина: 1.0
  Обоснование: Организация «Фабрика Счастье»

## Выводы

1. Поиск несколько улучшает результаты
2. Нужно сбрасывать индексы перед оценкой датасета
3. Нужно доработать выход модели строго в json с помощью Structured output. И langchain-ollama и langchain-deepseek поддержиают эту опцию, но нужно тестить
4. Поработать промптами над ограничением ризонинга

